In [2]:
!pip install bs4

In [1]:
!pip install milvus pymilvus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.64.1
    Uninstalling grpcio-1.64.1:
      Successfully uninstalled grpcio-1.64.1


In [3]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [4]:
!pip install rank_bm25

Importing the necessary libraries

In [117]:
import re
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from sentence_transformers import SentenceTransformer, util
import torch
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType, utility
from milvus import default_server
from rank_bm25 import BM25Okapi
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urldefrag
from collections import deque
import json
import socket
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


This function extracts main content from HTML using BeautifulSoup based on specific tags

In [ ]:
def extract_main_content(soup):

  main_content = soup.find('div', itemprop='articleBody')
  if not main_content:
    main_content = soup.find('article', id='contents')
    if not main_content:
      return ""

  for tag in main_content.find_all(['h1', 'h2', 'h3']):
    tag.decompose()

  return main_content.get_text(separator='\n', strip=True)

This function performs web crawling starting from a given URL up to a maximum depth.

In [ ]:
def crawl(start_url, max_depth=5):
  visited = set()
  queue = deque([(start_url, 1)])
  all_data = []

  while queue:
    url, depth = queue.popleft()
    if depth > max_depth or url in visited:
      continue
    visited.add(url)

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    page_text = extract_main_content(soup)

    page_data = {
      'url': url,
      'depth': depth,
      'text': page_text
    }

    all_data.append(page_data)

    links = soup.find_all('a', href=True)
    for link in links:
      sub_url = urljoin(url, link['href'])
      main_url, _ = urldefrag(sub_url)
      if main_url.startswith('https://docs.nvidia.com/cuda/') and main_url not in visited:
        queue.append((main_url, depth + 1))

  return all_data


In [ ]:
url = 'https://docs.nvidia.com/cuda/'
data = crawl(url, max_depth=5)

In [ ]:
with open('crawled_data.json', 'w', encoding='utf-8') as f:
  json.dump(data, f, ensure_ascii=False, indent=4)

In [8]:
with open('crawled_data.json', 'r', encoding='utf-8') as f:
  data = json.load(f)

In [9]:
print(data[7]['url'])
print(data[7]['depth'])
print(data[7]['text'][:2000])

https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html
2
CUDA C++ Programming Guide
The programming guide to the CUDA model and interface.
Changes from Version 12.4
Added section
Asynchronous Data Copies using Tensor Memory Access (TMA)
.
Added
Unified Memory Programming
guide supporting Grace Hopper with Address Translation Service (ATS) and Heterogeneous Memory Management (HMM ) on x86.
The Graphics Processing Unit (GPU)
1
provides much higher instruction throughput and memory bandwidth than the CPU within a similar price and power envelope. Many applications leverage these higher capabilities to run faster on the GPU than on the CPU (see
GPU Applications
). Other computing devices, like FPGAs, are also very energy efficient, but offer much less programming flexibility than GPUs.
This difference in capabilities between the GPU and the CPU exists because they are designed with different goals in mind. While the CPU is designed to excel at executing a sequence of operations, 

In [10]:
c=0
text1 = ""
for i in range(len(data)):
  if(data[i]['url'].startswith("https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/")):
    if(c==0):
      print(data[i]['url'])
      print()
      text1 = data[i]['text']
      break
    c+=1

https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html



In [11]:
text1

'CUDA C++ Best Practices Guide\nThe programming guide to using the CUDA Toolkit to obtain the best performance from NVIDIA GPUs.\nThis Best Practices Guide is a manual to help developers obtain the best performance from NVIDIA\n®\nCUDA\n®\nGPUs. It presents established parallelization and optimization techniques and explains coding metaphors and idioms that can greatly simplify programming for CUDA-capable GPU architectures.\nWhile the contents can be used as a reference manual, you should be aware that some topics are revisited in different contexts as various programming and configuration topics are explored. As a result, it is recommended that first-time readers proceed through the guide sequentially. This approach will greatly improve your understanding of effective programming practices and enable you to better use the guide for reference later.\nThe discussions in this guide all use the C++ programming language, so you should be comfortable reading C++ code.\nThis guide refers to

This function processes the extracted data by splitting into chunks, filtering based on length and content criteria, and preparing it for further processing.

In [12]:
def chunk_data(data):
  scraped_data = []
  c=0
  for i in range(len(data)):
    text1 = data[i]['text']
    # text1 = data[i]
    text1 = text1.split('\uf0c1')
    text1 = [item.split('.\n') for item in text1]
    text1 = [item for sublist in text1 for item in sublist]
    text1 = [item.replace('\n'," ") for item in text1]
    text1 = [re.sub(r'\s+', ' ', item) for item in text1]
    for text in text1:
      if text!='' and len(text.split())>10:
        scraped_data.append(text)

  scraped_data = [s for s in scraped_data if len(s) <= 2000 and len(s)>100]
  return scraped_data


In [13]:
scraped_data = chunk_data(data)

In [14]:
len(scraped_data)

53395

In [15]:
scraped_data = list(set(scraped_data))

In [16]:
len(scraped_data)

36033

In [17]:
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)
stemmer = PorterStemmer()

This function preprocesses text data by tokenizing, converting to lowercase, removing stopwords and punctuation, and stemming using NLTK.

In [18]:
def preprocess_text(data):
  p_data = []
  for text in data:
    tokens = word_tokenize(text)
    tokens = [token.lower() for token in tokens if token.lower() not in punctuation]
    tokens = [token for token in tokens if token.lower() not in stop_words]
    tokens = [stemmer.stem(token) for token in tokens]
    p_data.append(" ".join(tokens))

  return p_data


In [19]:
preprocessed_sentences = preprocess_text(scraped_data)

In [20]:
len(preprocessed_sentences)

36033

In [102]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [22]:
embeddings1 = model.encode(preprocessed_sentences, convert_to_tensor=False)
embeddings1.shape

(36033, 384)

In [23]:
embeddings1[0][:10]

array([ 0.11409242, -0.10919255, -0.10852886,  0.01782113, -0.0309688 ,
        0.00673167,  0.13329148,  0.12780659, -0.07151479,  0.09632704],
      dtype=float32)

In [24]:
np.save('embeddings.npy',embeddings1)

In [25]:
embeddings1 = np.load('embeddings.npy')

Checks if a server is running on a specified host and port using a socket connection.

In [26]:
def is_server_running(host, port):
  sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
  result = sock.connect_ex((host, port))
  sock.close()
  return result == 0

Connects to a Milvus server, starting it if necessary, and establishes a connection named "default".

In [27]:
def connect_to_milvus():
  # connections.disconnect("default")
  if not is_server_running(host="127.0.0.1",port=default_server.listen_port):
    default_server.start()
    connections.connect("default",host="127.0.0.1",port=default_server.listen_port)
  print("Connected to Milvus.")

In [28]:
connect_to_milvus()

Connected to Milvus.


Creates a Milvus collection with specific fields (id, index, text, embedding) and a defined schema.

In [29]:
def create_collection(collection_name, dim=384):

  fields = [
      FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
      FieldSchema(name="index", dtype=DataType.INT64),
      FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=2048),
      FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=dim)
  ]
  schema = CollectionSchema(fields, "CUDA documentation collection")
  collection = Collection(collection_name, schema)
  print(f"Collection schema: {collection.schema}")
  return collection

In [30]:
if not utility.has_collection("cuda_docs_collection"):
  collection = create_collection("cuda_docs_collection")
# collection.drop()

Collection schema: {'auto_id': True, 'description': 'CUDA documentation collection', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'index', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 2048}}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}], 'enable_dynamic_field': False}


Inserts scraped data and corresponding embeddings into the Milvus collection in batches, flushing data afterward.

In [31]:
def create_index(collection, field_name="embedding"):
  index= {"metric_type": "COSINE", "index_type": "IVF_FLAT", "params": {"nlist": 128}}
  collection.create_index(field_name, index)
  print(f"Index created on field '{field_name}' successfully!")

In [32]:
create_index(collection)

Index created on field 'embedding' successfully!


Inserts scraped data and corresponding embeddings into the Milvus collection in batches, flushing data afterward.

In [33]:
def insert_data(collection,scraped_data,embeddings):
  batch_size = 1000
  for i in range(0, len(scraped_data), batch_size):
    end = min(i + batch_size, len(scraped_data))
    batch_entities = [
      list(range(i,end)),
      [text for text in scraped_data[i:end]],
      [embedding for embedding in embeddings[i:end]]
      ]
    collection.insert(batch_entities)
  collection.flush()
  print("Data inserted successfully!")

In [34]:
if collection.is_empty:
  insert_data(collection,scraped_data,embeddings1)

Data inserted successfully!


In [35]:
collection.num_entities

36033

Normalizes distances and scores of entities retrieved from Milvus to a common scale.



In [36]:
def normalize_distances(entities):
  distances = [entity[1].distance for entity in entities]
  scores = [entity[0] for entity in entities]
  min_dist = min(distances)
  max_dist = max(distances)

  min_score = min(scores)
  max_score = max(scores)

  normalized_distances = [(dist - min_dist) / (max_dist - min_dist) for dist in distances]
  normalized_scores = [(score - min_score) / (max_score - min_score) for score in scores]

  for i, entity in enumerate(entities):
    entity[1].distance = normalized_distances[i]
    entity[0] = normalized_scores[i]

  return entities

Performs hybrid retrieval combining BM25 scoring and BERT-based embedding search. It ranks results based on a weighted combination of BM25 scores and normalized distances from Milvus

In [37]:
def hybrid_retrieval(query_inp,collection,model,preprocessed_sentences,scraped_data,alpha=0.5):
  tokenized_corpus = [text.split() for text in preprocessed_sentences]
  bm25 = BM25Okapi(tokenized_corpus)

  tokenized_query = query_inp[0].split()
  doc_scores = bm25.get_scores(tokenized_query)

  doc_scores1 = []
  dict_scores = {}
  for i,v in enumerate(doc_scores):
    doc_scores1.append((i,v))
    dict_scores[i]=v

  doc_scores1_sorted = sorted(doc_scores1, key=lambda x: x[1],reverse=True)[:300]

  # docs = bm25.get_top_n(tokenized_query, list(scraped_data), n=100)
  # print(docs)
  query_embedding = model.encode(query_inp, convert_to_tensor=False)
  collection.load()
  bert_result = collection.search(query_embedding,"embedding",param={"metric_type":"COSINE", "params":{"nprobe":128}},limit=1500,output_fields=["index","text","embedding"])

  bert_results_final = [r.entity.get('index') for r in bert_result[0]]
  scores_final = []
  for i in doc_scores1_sorted:
    if(i[0] in bert_results_final):
      scores_final.append(i[0])

  bert_results1 = []
  for r in bert_result[0]:
    if r.entity.get('index') in scores_final:
      bert_results1.append([dict_scores[r.entity.get('index')],r])

  bert_results1 = normalize_distances(bert_results1)
  for i in range(len(bert_results1)):
    bert_results1[i][0] = alpha*bert_results1[i][0]+(1-alpha)*bert_results1[i][1].distance

  bert_results_final = sorted(bert_results1,key=lambda x: x[0],reverse=True)
  bert_results_final = [(r[0],r[1].entity.get('text')) for r in bert_results_final]
  return bert_results_final

In [103]:
user_query = "What is libdevice library?"
query_inp = preprocess_text([user_query])
retrieved_docs = hybrid_retrieval(query_inp,collection,model,preprocessed_sentences,scraped_data,alpha=0.6)

In [104]:
retrieved_docs[:5]

[(0.8351091601880062,
  "This guide documents both the functions available in libdevice and the basic usage of the library from a compiler writer's perspective."),
 (0.6986700025528929,
  'libdevice User’s Guide The libdevice library is an LLVM bitcode library that implements common functions for GPU kernels'),
 (0.5952769102537547,
  'The libdevice library ships as an LLVM bitcode library and is meant to be linked with the target module early in the compilation process. The standard process for linking with libdevice is to first link it with the target module, then run the standard LLVM optimization and code generation passes. This allows the optimizers to inline and perform analyses on the used library functions, and eliminate any used functions as dead code'),
 (0.40148803680612105,
  'cusolver library provides dynamic library libcusolver.so and static library libcusolver_static.a . If the user links the application with libcusolver.so , libcublas.so , libcublasLt.so and libcusparse

In [105]:
for i in retrieved_docs[:5]:
  print(i[1],end="\n\n")

This guide documents both the functions available in libdevice and the basic usage of the library from a compiler writer's perspective.

libdevice User’s Guide The libdevice library is an LLVM bitcode library that implements common functions for GPU kernels

The libdevice library ships as an LLVM bitcode library and is meant to be linked with the target module early in the compilation process. The standard process for linking with libdevice is to first link it with the target module, then run the standard LLVM optimization and code generation passes. This allows the optimizers to inline and perform analyses on the used library functions, and eliminate any used functions as dead code

cusolver library provides dynamic library libcusolver.so and static library libcusolver_static.a . If the user links the application with libcusolver.so , libcublas.so , libcublasLt.so and libcusparse.so are also required. If the user links the application with libcusolver_static.a , the following librarie

Merging the retrieved con

In [106]:
context = [doc[1] for doc in retrieved_docs[:5]]
context = "\n".join(context)

In [107]:
context

"This guide documents both the functions available in libdevice and the basic usage of the library from a compiler writer's perspective.\nlibdevice User’s Guide The libdevice library is an LLVM bitcode library that implements common functions for GPU kernels\nThe libdevice library ships as an LLVM bitcode library and is meant to be linked with the target module early in the compilation process. The standard process for linking with libdevice is to first link it with the target module, then run the standard LLVM optimization and code generation passes. This allows the optimizers to inline and perform analyses on the used library functions, and eliminate any used functions as dead code\ncusolver library provides dynamic library libcusolver.so and static library libcusolver_static.a . If the user links the application with libcusolver.so , libcublas.so , libcublasLt.so and libcusparse.so are also required. If the user links the application with libcusolver_static.a , the following librari

In [108]:
model2 = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer2 = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


This functions takes the user_query, ranked output and bert model to produce the relevant answer for the query

In [121]:
def question_answer(user_query, context, model, tokenizer):

    inputs = tokenizer.encode_plus(user_query, context, return_tensors='pt', add_special_tokens=True)
    input_ids = inputs['input_ids']
    token_type_ids = inputs['token_type_ids']

    outputs = model(input_ids, token_type_ids=token_type_ids)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    answer_start = torch.argmax(start_logits)
    answer_end = torch.argmax(end_logits)

    answer_tokens = input_ids[0][answer_start:answer_end + 1]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

    corrected_answer = answer.capitalize()

    return corrected_answer

In [122]:
answer = question_answer(user_query,context,model2,tokenizer2)
answer

'A collection of nvvm bitcode functions that implement common functions for nvidia gpu devices'